<a href="https://colab.research.google.com/github/Nisasoylu/BERT2OME/blob/main/BERT_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importings:

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import RandomSearch
from sklearn.metrics import precision_recall_curve
from tensorflow.keras.layers import Dense, Activation, Conv1D, ZeroPadding1D, MaxPooling1D, Dropout, Flatten, Conv2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, precision_recall_curve, auc, roc_curve
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from xgboost import XGBClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE
from kerastuner import RandomSearch
from sklearn.metrics import confusion_matrix
import torch
from transformers import BertTokenizer, BertModel

<ipython-input-4-ec8748ad9989>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [ ]:
def draw_roc_pr_curve(fpr_keras, tpr_keras, auc_keras, recall, precision, auc_precision_recall, model_name):

  # ROC Curve
  plt.plot([0, 1], [0, 1], "k--")
  plt.plot(fpr_keras, tpr_keras, label="AUC (area = {:.3f})".format(auc_keras))
  plt.xlabel("False positive rate", fontsize=15)
  plt.ylabel("True positive rate", fontsize=15)
  plt.title(f"ROC curve for {model_name}", fontsize=20)
  plt.legend(loc='best', prop={'size': 15})
  plt.show()

  # Precision-Recall Curve
  plt.plot([0, 1], [0, 1], 'k--')
  plt.plot(recall, precision, label="AUC (area = {:.3f})".format(auc_precision_recall))
  plt.xlabel("Recall", fontsize=15)
  plt.ylabel("Precision", fontsize=15)
  plt.title(f"PR curve for {model_name}", fontsize=20)
  plt.legend(loc="best", prop={'size': 15})
  plt.show()

# BASELINE MODELS

Dataset reading for baseline models:

In [ ]:
################### HUMAN1 Dataset #######################
test_neg = pd.read_csv(r"test_new_neg_H_N", header = None)
test_pos = pd.read_csv(r"test_new_pos_H_P", header = None)
train_neg = pd.read_csv(r"train_new_neg_H_N", header = None)
train_pos = pd.read_csv(r"train_new_pos_H_P", header = None)

# Features
x = pd.concat([train_pos, train_neg, test_pos, test_neg], ignore_index=True)

# Labels
train_pos_labels = [1] * len(train_pos)
train_neg_labels = [0] * len(train_neg)
test_pos_labels = [1] * len(test_pos)
test_neg_labels = [0] * len(test_neg)

y = train_pos_labels + train_neg_labels + test_pos_labels + test_neg_labels
test_neg.head()
###########################################################

,0
0,G G C G C A A U C G G U U A G C G C G U U C G ...
1,C C C A C G C G G G A G A C C C G G G U U C A ...
2,G G C C A U G U G C U C A U U U C C A C U U A ...
3,U U A A U C U C A G G G U C G U G G G U U C G ...
4,G U G G C C G C A G C A A C C U C G G U U C G ...


In [ ]:
################### HUMAN2 Dataset #######################
rmbase_dataset_ = pd.read_excel("RMBase_800_CD-HIT.xlsx")
y = rmbase_dataset_.label
x = rmbase_dataset_.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

770
G U C G G A A G C A G G G C C U G G C U C A G C A C C G G G A G G G C G C C G C C


In [ ]:
################### MOUSE Dataset #######################
M = pd.read_excel("M.musculus.xlsx")
y = M.label
y = y.to_list()
x = M.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

44
A A U G G U U A G C A C U C U G G A C U U U G A A U C C A G C A A U C U G A G U U 


In [ ]:
################### YEAST Dataset #######################
S = pd.read_excel("S.cerevisiae.xlsx")
y = S.label
x = S.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

278
A A G A C A U C A A U U A G U A C G G G C G U G U G G U C U A G U G G U A U G A U 


In [ ]:
# Number of positive and negative instances
#y_array = np.array(y)
#counts = np.bincount(y_array)
#print(f"Number of 1s: {counts[1]}")
#print(f"Number of 0s: {counts[0]}")

One-hot representation:

In [ ]:
def one_hot_coversion(each_row):
    nuc_vector = []
    nuc_vector = np.array(nuc_vector)

    for i in each_row:
        if i == "A":
            nuc_vector = np.concatenate((nuc_vector, [1, 0, 0, 0]))
        elif i == "G":
            nuc_vector = np.concatenate((nuc_vector, [0, 1, 0, 0]))
        elif i == "C":
            nuc_vector = np.concatenate((nuc_vector, [0, 0, 1, 0]))
        elif i == "U":
            nuc_vector = np.concatenate((nuc_vector, [0, 0, 0, 1]))

    return nuc_vector

all_vectors = []
for i in range(0, len(x)):
    row_vector = one_hot_coversion(x[0][i])
    all_vectors.append(row_vector)

new_x = pd.DataFrame(all_vectors)

In [ ]:
new_x.head()

,0,1,2,3,4,5,6,7,8,9,...,154,155,156,157,158,159,160,161,162,163
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


SMOTE Part:

In [ ]:
counter = Counter(y)
print(counter)

oversample = SMOTE()
new_x, y = oversample.fit_resample(new_x, y)

counter = Counter(y)
print("After SMOTE:")
print(counter)

Counter({1: 470, 0: 300})
After SMOTE:
Counter({1: 470, 0: 470})


## Decision Tree:

In [ ]:
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier(random_state = 0)
xtrain, xtest, ytrain, ytest = train_test_split(new_x, y, test_size = 0.20, random_state=42)

dt.fit(xtrain, ytrain)
decision_tree_predictions = dt.predict(xtest)
dt_predictions_proba = dt.predict_proba(xtest)
dt_predictions_proba = dt_predictions_proba[:,1]

print("predictions:", decision_tree_predictions)
print("**********")
print("prediction probabilities:", dt_predictions_proba)

print("Model accuracy score: " + str(accuracy_score(ytest, decision_tree_predictions)))
f1_score_value = f1_score(ytest, decision_tree_predictions)
print("f1_score:", f1_score_value)
print("Precision:", metrics.precision_score(ytest, decision_tree_predictions))
print("Recall:", metrics.recall_score(ytest, decision_tree_predictions))

confusion_matrix = confusion_matrix(ytest, decision_tree_predictions)
sns.heatmap(confusion_matrix, annot = True, fmt = "d", cbar = False)
plt.title("Decision Tree Confusion Matrix", fontsize = 20)
plt.show()

#fpr_keras_dt, tpr_keras_dt, thresholds_keras_dt = roc_curve(ytest, decision_tree_predictions)
fpr_keras_dt, tpr_keras_dt, thresholds_keras_dt = roc_curve(ytest, dt_predictions_proba)
auc_keras_dt = auc(fpr_keras_dt, tpr_keras_dt)
print("ROC AUC Score:", auc_keras_dt)

#precision_dt, recall_dt, _ = precision_recall_curve(ytest, decision_tree_predictions)
precision_dt, recall_dt, _ = precision_recall_curve(ytest, dt_predictions_proba)
auc_precision_recall_dt = auc(recall_dt, precision_dt)
print("Precision-Recall AUC Score:", auc_precision_recall_dt)

draw_roc_pr_curve(fpr_keras_dt, tpr_keras_dt, auc_keras_dt,
                  recall_dt, precision_dt, auc_precision_recall_dt,
                  model_name = "Decision Tree")

## Random Forest:

In [ ]:
from sklearn.metrics import confusion_matrix

rfc = RandomForestClassifier(random_state=0, criterion="entropy")
xtrain, xtest, ytrain, ytest = train_test_split(new_x, y, test_size = 0.20, random_state=42)

rfc.fit(xtrain, ytrain)
random_forest_predictions = rfc.predict(xtest)
rf_predictions_proba = rfc.predict_proba(xtest)
rf_predictions_proba = rf_predictions_proba[:,1]

print("Model accuracy score: " + str(accuracy_score(ytest, random_forest_predictions)))
print("Precision:", metrics.precision_score(ytest, random_forest_predictions))
print("Recall:", metrics.recall_score(ytest, random_forest_predictions))

confusion_matrix_rf = confusion_matrix(ytest, random_forest_predictions)
sns.heatmap(confusion_matrix_rf, annot=True, fmt="d", cbar=False)
plt.title("Random Forest Confusion Matrix", fontsize=20)
plt.show()

fpr_keras_rf, tpr_keras_rf, thresholds_keras_rf = roc_curve(ytest, rf_predictions_proba)
#fpr_keras_rf, tpr_keras_rf, thresholds_keras_rf = roc_curve(ytest, random_forest_predictions)
auc_keras_rf = auc(fpr_keras_rf, tpr_keras_rf)
print("ROC AUC Score:", auc_keras_rf)

precision_rf, recall_rf, _ = precision_recall_curve(ytest, rf_predictions_proba)
#precision_rf, recall_rf, _ = precision_recall_curve(ytest, random_forest_predictions)
auc_precision_recall_rf = auc(recall_rf, precision_rf)
print("Precision-Recall AUC Score:", auc_precision_recall_rf)

draw_roc_pr_curve(fpr_keras_rf, tpr_keras_rf, auc_keras_rf,
                  recall_rf, precision_rf, auc_precision_recall_rf,
                  model_name = "Random Forest")

## XGBoost:

In [ ]:
from sklearn.metrics import confusion_matrix

xgmodel = XGBClassifier()
xtrain, xtest, ytrain, ytest = train_test_split(new_x, y, test_size = 0.20, random_state=42)

xgmodel.fit(xtrain, ytrain, verbose=False)
xgboost_predictions = xgmodel.predict(xtest)
xgb_predictions_proba = xgmodel.predict_proba(xtest)
xgb_predictions_proba = xgb_predictions_proba[:,1]

print("Model accuracy score: " + str(accuracy_score(ytest, xgboost_predictions)))
print("f1 score: ", metrics.f1_score(ytest, xgboost_predictions))
print("Precision:", metrics.precision_score(ytest, xgboost_predictions))
print("Recall:", metrics.recall_score(ytest, xgboost_predictions))

confusion_matrix_xgb = confusion_matrix(ytest, xgboost_predictions)
sns.heatmap(confusion_matrix_xgb, annot=True, fmt="d", cbar=False)
plt.title("XGBoost Confusion Matrix", fontsize=20)
plt.show()

#fpr_keras_xgb, tpr_keras_xgb, thresholds_keras_xgb = roc_curve(ytest, xgboost_predictions)
fpr_keras_xgb, tpr_keras_xgb, thresholds_keras_xgb = roc_curve(ytest, xgb_predictions_proba)
auc_keras_xgb = auc(fpr_keras_xgb, tpr_keras_xgb)
print("ROC AUC Score:", auc_keras_xgb)

precision_xgb, recall_xgb, _ = precision_recall_curve(ytest, xgb_predictions_proba)
#precision_xgb, recall_xgb, _ = precision_recall_curve(ytest, xgboost_predictions)
auc_precision_recall_xgb = auc(recall_xgb, precision_xgb)
print("Precision-Recall AUC Score:", auc_precision_recall_xgb)

draw_roc_pr_curve(fpr_keras_xgb, tpr_keras_xgb, auc_keras_xgb,
                  recall_xgb, precision_xgb, auc_precision_recall_xgb,
                  model_name = "XGBoost")

# Deep Learning Models

Dataset reading for DL models:

In [ ]:
################### HUMAN1 Dataset #######################
x = np.load("BERTHUMAN1EMBEDDINGSX.npy")
y = np.load("BERTHUMAN1EMBEDDINGSY.npy")
###########################################################

In [ ]:
################### HUMAN2 Dataset #######################
x = np.load("BERTHUMAN2EMBEDDINGSX.npy")
y = np.load("BERTHUMAN2EMBEDDINGSY.npy")
###########################################################

In [ ]:
################### MOUSE Dataset #######################
x = np.load("BERTMOUSEEMBEDDINGSX.npy")
y = np.load("BERTMOUSEEMBEDDINGSY.npy")
###########################################################

In [ ]:
################### YEAST Dataset #######################
x = np.load("BERTYEASTEMBEDDINGSX.npy")
y = np.load("BERTYEASTEMBEDDINGSY.npy")
###########################################################

Model setups:

In [ ]:
# BERT
MODEL_NAME = "BERT"
EMBEDDING_LENGTH = 768

## Embeddings + Random Forest:

In [ ]:
from sklearn.metrics import confusion_matrix

rfc = RandomForestClassifier(random_state=0, criterion="entropy")
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=42)

rfc.fit(xtrain, ytrain)
bert_rf_predictions = rfc.predict(xtest)
bert_rf_predictions_proba = rfc.predict_proba(xtest)
bert_rf_predictions_proba = bert_rf_predictions_proba[:,1]

print("Model accuracy score: " + str(accuracy_score(ytest, bert_rf_predictions)))
print("Precision:", metrics.precision_score(ytest, bert_rf_predictions))
print("Recall:", metrics.recall_score(ytest, bert_rf_predictions))

confusion_matrix_rf = confusion_matrix(ytest, bert_rf_predictions)
sns.heatmap(confusion_matrix_rf, annot = True, fmt = "d", cbar = False)
plt.title(f"{MODEL_NAME} + Random Forest Confusion Matrix", fontsize = 20)
plt.show()

fpr_keras_bert_rf, tpr_keras_bert_rf, thresholds_keras_bert_rf = roc_curve(ytest, bert_rf_predictions_proba)
#fpr_keras_bert_rf, tpr_keras_bert_rf, thresholds_keras_bert_rf = roc_curve(ytest, bert_rf_predictions)
auc_keras_bert_rf = auc(fpr_keras_bert_rf, tpr_keras_bert_rf)
print("AUC Score:", auc_keras_bert_rf)

precision_bert2_rf, recall_bert2_rf, _ = precision_recall_curve(ytest, bert_rf_predictions_proba)
#precision_bert2_rf, recall_bert2_rf, _ = precision_recall_curve(ytest, bert_rf_predictions)
auc_precision_recall_bert2_rf = auc(recall_bert2_rf, precision_bert2_rf)
print("Precision-Recall AUC Score:", auc_precision_recall_bert2_rf)

draw_roc_pr_curve(fpr_keras_bert_rf, tpr_keras_bert_rf, auc_keras_bert_rf,
                  recall_bert2_rf, precision_bert2_rf, auc_precision_recall_bert2_rf,
                  model_name = MODEL_NAME + "+ Random Forest")

## Embedding + XGBoost:

In [ ]:
from sklearn.metrics import confusion_matrix

xgmodel = XGBClassifier()
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.20, random_state = 42)

xgmodel.fit(xtrain, ytrain, verbose=False)
bert_xgb_predictions = xgmodel.predict(xtest)
bert_xgb_predictions_proba = xgmodel.predict_proba(xtest)
bert_xgb_predictions_proba = bert_xgb_predictions_proba[:,1]

print("Model accuracy score: " + str(accuracy_score(ytest, bert_xgb_predictions)))
print("f1 score: ", metrics.f1_score(ytest, bert_xgb_predictions))
print("Precision:", metrics.precision_score(ytest, bert_xgb_predictions))
print("Recall:", metrics.recall_score(ytest, bert_xgb_predictions))

confusion_matrix = confusion_matrix(ytest, bert_xgb_predictions)
sns.heatmap(confusion_matrix, annot = True, fmt = "d", cbar = False)
plt.title("BERT + XGBoost Confusion Matrix", fontsize = 20)
plt.show()

#fpr_keras_bert_xgb, tpr_keras_bert_xgb, thresholds_keras_bert_xgb = roc_curve(ytest, bert_xgb_predictions)
fpr_keras_bert_xgb, tpr_keras_bert_xgb, thresholds_keras_bert_xgb = roc_curve(ytest, bert_xgb_predictions_proba)
auc_keras_bert_xgb = auc(fpr_keras_bert_xgb, tpr_keras_bert_xgb)
print("AUC Score:", auc_keras_bert_xgb)

precision_bert2_xgb, recall_bert2_xgb, _ = precision_recall_curve(ytest, bert_xgb_predictions_proba)
#precision_bert2_xgb, recall_bert2_xgb, _ = precision_recall_curve(ytest, bert_xgb_predictions)
auc_precision_recall_bert2_xgb = auc(recall_bert2_xgb, precision_bert2_xgb)
print("Precision-Recall AUC Score:", auc_precision_recall_bert2_xgb)

draw_roc_pr_curve(fpr_keras_bert_xgb, tpr_keras_bert_xgb, auc_keras_bert_xgb,
                  recall_bert2_xgb, precision_bert2_xgb, auc_precision_recall_bert2_xgb,
                  model_name = MODEL_NAME + "+ XGBoost")

## Embedding + 1D-CNN:

In [ ]:
from sklearn.metrics import confusion_matrix

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.1, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

def CNN_1D():
    model = Sequential()
    # layer 1
    model.add(Conv1D(32, 3, input_shape=(43 * EMBEDDING_LENGTH, 1), activation="relu"))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.1))
    # layer 2
    model.add(Conv1D(16, 3, activation="relu"))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.2))
    # Flattening Layer:
    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    # Last Layer:
    model.add(Dense(2, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam",
                  metrics=["accuracy", "mse", "mape", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

model1 = CNN_1D()

reshaped_x_train = np.asarray(x_train).reshape(len(np.asarray(x_train)),43*EMBEDDING_LENGTH,1)
reshaped_x_val = np.asarray(x_val).reshape(len(np.asarray(x_val)),43*EMBEDDING_LENGTH,1)

history1_ = model1.fit(reshaped_x_train, utils.to_categorical(y_train,2),
                       validation_data=(reshaped_x_val, utils.to_categorical(y_val,2)),
                       epochs=25, batch_size=20, verbose=1)    # epoch = 15

plt.plot(history1_.history["accuracy"])
plt.plot(history1_.history["val_accuracy"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.title("1D CNN Model Accuracy", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

plt.plot(history1_.history["loss"])
plt.plot(history1_.history["val_loss"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Loss", fontsize=10)
plt.title("1D CNN Model Loss", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

print(model1.summary())

cnn_predictions1 = model1.predict(x_test)
bert2_probs_cnn1 = cnn_predictions1[:,1]
cnn_predictions1 = np.argmax(cnn_predictions1, axis = 1)

print("Model accuracy score: " + str(accuracy_score(y_test, cnn_predictions1)))
print("f1 score: ", metrics.f1_score(y_test, cnn_predictions1))
print("Precision:", metrics.precision_score(y_test, cnn_predictions1))
print("Recall:", metrics.recall_score(y_test, cnn_predictions1))

confusion_matrix = confusion_matrix(y_test, cnn_predictions1)
sns.heatmap(confusion_matrix, annot = True, fmt = "d", cbar = False)
plt.title("BERT + 1D CNN Confusion Matrix", fontsize = 20)
plt.show()

fpr_keras_bert_cnn1_hyp, tpr_keras_bert_cnn1_hyp, _ = roc_curve(y_test, bert2_probs_cnn1)
auc_keras_bert_cnn1_hyp = auc(fpr_keras_bert_cnn1_hyp, tpr_keras_bert_cnn1_hyp)
print("ROC AUC Score:", auc_keras_bert_cnn1_hyp)

precision_bert2_cnn1, recall_bert2_cnn1, _ = precision_recall_curve(y_test, bert2_probs_cnn1)
auc_precision_recall_bert2_cnn1 = auc(recall_bert2_cnn1, precision_bert2_cnn1)
print("Precision-Recall AUC Score:", auc_precision_recall_bert2_cnn1)

draw_roc_pr_curve(fpr_keras_bert_cnn1_hyp, tpr_keras_bert_cnn1_hyp, auc_keras_bert_cnn1_hyp,
                  recall_bert2_cnn1, precision_bert2_cnn1, auc_precision_recall_bert2_cnn1,
                  model_name = MODEL_NAME + "+ 1D-CNN")

## Embedding + 2D-CNN:

In [ ]:
# With Keras Tuner

In [ ]:
project_name = "BEST_PARAMETERS_HUMAN1_BERT"
#project_name = "BEST_PARAMETERS_HUMAN2_BERT"
#project_name = "BEST_PARAMETERS_MOUSE_BERT"
#project_name = "BEST_PARAMETERS_YEAST_BERT"

In [ ]:
from sklearn.metrics import confusion_matrix

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.1, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(filters=hp.Int("conv_1_filter", min_value=10, max_value=32, step=4),
            kernel_size=hp.Choice("conv_1_kernel", values=[3, 7]), activation="relu", input_shape=(EMBEDDING_LENGTH, 43, 1)),

        keras.layers.Conv2D(filters=hp.Int("conv_2_filter", min_value=32, max_value=64, step=4),
            kernel_size=hp.Choice("conv_2_kernel", values=[3, 6]), activation="relu"),

        keras.layers.Flatten(),
        keras.layers.Dense(units=hp.Int("dense_1_units", min_value=32, max_value=80, step=4), activation="relu"),
        keras.layers.Dense(2, activation="softmax")
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 5e-5])),
                  loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.summary()
    return model

tuner = RandomSearch(build_model, objective = "val_accuracy", max_trials = 10, directory = "output", project_name = project_name)

# project_name = BERT_HUMAN1_BEST_PARAMETERS_

reshaped_x_train = np.asarray(x_train).reshape(len(np.asarray(x_train)),EMBEDDING_LENGTH,43,1)
reshaped_x_val = np.asarray(x_val.reshape(len(np.asarray(x_val)),EMBEDDING_LENGTH,43,1))

tuner.search(reshaped_x_train, utils.to_categorical(y_train,2),
             validation_data = (reshaped_x_val, utils.to_categorical(y_val,2)), epochs = 20)

model_2D_ht = tuner.get_best_models(num_models=1)[0]

In [ ]:
from sklearn.metrics import confusion_matrix

history3 = model_2D_ht.fit(np.asarray(x_train).reshape(len(x_train), EMBEDDING_LENGTH, 43, 1), utils.to_categorical(y_train, 2),
                          validation_data=(np.asarray(x_val.reshape(len(x_val), EMBEDDING_LENGTH, 43, 1)), utils.to_categorical(y_val, 2)),
                          epochs=20, batch_size=20, verbose=1)

plt.plot(history3.history["accuracy"])
plt.plot(history3.history["val_accuracy"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.title("2D CNN Model Accuracy", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

plt.plot(history3.history["loss"])
plt.plot(history3.history["val_loss"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Loss", fontsize=10)
plt.title("2D CNN Model Loss", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

print(model_2D_ht.summary())

cnn_predictions3 = model_2D_ht.predict(np.asarray(x_test).reshape(len(np.asarray(x_test)), EMBEDDING_LENGTH, 43, 1))
bert2_probs = cnn_predictions3[:, 1]
cnn_predictions3 = np.argmax(cnn_predictions3, axis=1)

print("Model accuracy score: " + str(accuracy_score(y_test, cnn_predictions3)))
print("f1 score: ", metrics.f1_score(y_test, cnn_predictions3))
print("Precision:", metrics.precision_score(y_test, cnn_predictions3))
print("Recall:", metrics.recall_score(y_test, cnn_predictions3))

conf_matrix = confusion_matrix(y_test, cnn_predictions3)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cbar=False)
plt.title("BERT + 2D CNN + H. Tuning Confusion Matrix", fontsize=20)
plt.show()

print(cnn_predictions3)

fpr_keras_bert_cnn2_hyp, tpr_keras_bert_cnn2_hyp, thresholds_keras = roc_curve(y_test, bert2_probs)
auc_keras_bert_cnn2_hyp = auc(fpr_keras_bert_cnn2_hyp, tpr_keras_bert_cnn2_hyp)
print("ROC AUC Score:", auc_keras_bert_cnn2_hyp)

precision_bert2, recall_bert2, _ = precision_recall_curve(y_test, bert2_probs)
auc_precision_recall_bert2 = auc(recall_bert2, precision_bert2)
print("Precision-Recall AUC Score:", auc_precision_recall_bert2)

draw_roc_pr_curve(fpr_keras_bert_cnn2_hyp, tpr_keras_bert_cnn2_hyp, auc_keras_bert_cnn2_hyp,
                  recall_bert2, precision_bert2, auc_precision_recall_bert2,
                  model_name = MODEL_NAME + "+ 2D-CNN")

## BERT2OME with Chemical Properties PART:

Select dataset (original):

In [ ]:
################### HUMAN1 Dataset #######################
test_neg = pd.read_csv(r"test_new_neg_H_N", header = None)
test_pos = pd.read_csv(r"test_new_pos_H_P", header = None)
train_neg = pd.read_csv(r"train_new_neg_H_N", header = None)
train_pos = pd.read_csv(r"train_new_pos_H_P", header = None)

# Features
x_values = pd.concat([train_pos, train_neg, test_pos, test_neg], ignore_index=True)

# Labels
train_pos_labels = [1] * len(train_pos)
train_neg_labels = [0] * len(train_neg)
test_pos_labels = [1] * len(test_pos)
test_neg_labels = [0] * len(test_neg)

y_values = train_pos_labels + train_neg_labels + test_pos_labels + test_neg_labels
###########################################################

In [ ]:
################### HUMAN2 Dataset #######################
rmbase_dataset_ = pd.read_excel("RMBase_800_CD-HIT.xlsx")
y_values = rmbase_dataset_.label
x_values = rmbase_dataset_.drop(["label"], axis = 1)
print(len(x_values))
print(x_values[0][0])
###########################################################

770
G U C G G A A G C A G G G C C U G G C U C A G C A C C G G G A G G G C G C C G C C


In [ ]:
################### MOUSE Dataset #######################
M = pd.read_excel("M.musculus.xlsx")
y = M.label
y_values = y.to_list()
x_values = M.drop(["label"], axis = 1)
print(len(x_values))
print(x_values[0][0])
###########################################################

In [ ]:
################### YEAST Dataset #######################
S = pd.read_excel("S.cerevisiae.xlsx")
y_values = S.label
x_values = S.drop(["label"], axis = 1)
print(len(x_values))
print(x_values[0][0])
###########################################################

278
A A G A C A U C A A U U A G U A C G G G C G U G U G G U C U A G U G G U A U G A U 


Select dataset (Embeddings):

In [ ]:
################### HUMAN1 Dataset #######################
dataset_x = np.load("BERTHUMAN1EMBEDDINGSX.npy")
y = np.load("BERTHUMAN1EMBEDDINGSY.npy")
###########################################################

In [ ]:
################### HUMAN2 Dataset #######################
dataset_x = np.load("BERTHUMAN2EMBEDDINGSX.npy")
y = np.load("BERTHUMAN2EMBEDDINGSY.npy")
###########################################################

In [ ]:
################### MOUSE Dataset #######################
dataset_x = np.load("BERTMOUSEEMBEDDINGSX.npy")
y = np.load("BERTMOUSEEMBEDDINGSY.npy")
###########################################################

In [ ]:
################### YEAST Dataset #######################
dataset_x = np.load("BERTYEASTEMBEDDINGSX.npy")
y = np.load("BERTYEASTEMBEDDINGSY.npy")
###########################################################

In [ ]:
dataset_x = pd.DataFrame(dataset_x)

Select which location you will contain your model's best parameters:

In [ ]:
#project_name = "BEST_PARAMETERS_HUMAN1_CH_BERT"
#project_name = "BEST_PARAMETERS_HUMAN2_CH_BERT"
#project_name = "BEST_PARAMETERS_MOUSE_CH_BERT"
#project_name = "BEST_PARAMETERS_YEAST_CH_BERT"

In [ ]:
43*768

33024

In [ ]:
dataset_x.head()

,0,1,2,3,4,5,6,7,8,9,...,44022,44023,44024,44025,44026,44027,44028,44029,44030,44031
0,-0.016333,0.030015,-0.111715,-0.139209,0.128241,0.073813,-0.042372,-0.076964,0.007215,0.098894,...,0.064585,-0.054929,-0.007589,-0.106296,0.003818,0.004630,0.036363,-0.059417,-0.096753,0.115043
1,-0.088772,-0.064287,-0.058297,-0.213957,-0.001278,-0.030998,-0.202203,-0.236997,0.045164,0.157874,...,0.051153,0.120054,-0.015608,-0.126160,0.211296,-0.055563,-0.029729,0.175891,-0.076758,-0.008087
2,-0.010698,0.063155,-0.126877,-0.122769,0.094866,0.038334,-0.028891,-0.076412,-0.015086,0.100730,...,0.031660,-0.031144,-0.000875,-0.119689,0.026145,0.016671,0.052288,-0.043501,-0.126030,0.078901
3,-0.010015,-0.167163,-0.048913,-0.241675,-0.153408,-0.030678,-0.204013,-0.248120,-0.003481,0.181270,...,0.131978,0.142383,-0.018785,-0.127472,0.176770,-0.020787,-0.171453,0.205280,-0.127785,-0.053986
4,0.004464,0.025306,-0.134299,-0.116793,0.108124,0.084962,-0.029363,-0.064316,-0.002252,0.084748,...,0.068982,-0.060105,-0.008551,-0.104669,0.003502,0.012163,0.035501,-0.046423,-0.095274,0.108861


Adding chemical property representation:

In [ ]:
# A -> (1,1,1)  C -> (0,1,0)  G -> (1,0,0)  U -> (0,0,1)

def convert_chemical_property(each_row):
    chemical_vector = []
    chemical_vector = np.array(chemical_vector)

    for i in each_row:
        if i == "A":
            chemical_vector = np.concatenate((chemical_vector, [1, 1, 1]))
        elif i == "C":
            chemical_vector = np.concatenate((chemical_vector, [0, 1, 0]))
        elif i == "G":
            chemical_vector = np.concatenate((chemical_vector, [1, 0, 0]))
        elif i == "U":
            chemical_vector = np.concatenate((chemical_vector, [0, 0, 1]))

    return chemical_vector

all_chemical_vectors = []
for i in range(0, len(x_values)):
    row_vector = convert_chemical_property(x_values[0][i])
    all_chemical_vectors.append(row_vector)

chemical_dataframe = pd.DataFrame(all_chemical_vectors)
#chemical_dataframe.head()

In [ ]:
chemical_dataset_direct_concat = pd.concat([dataset_x, chemical_dataframe], ignore_index=True, axis=1)

dataframe_padding = pd.DataFrame(np.zeros(len(chemical_dataframe)))

dataframe_padding = pd.concat([dataframe_padding, dataframe_padding, dataframe_padding], ignore_index=True, axis=1)

chemical_dataset = pd.concat([dataframe_padding, chemical_dataframe, dataframe_padding], ignore_index=True, axis=1)

big_chemical_dataset = pd.concat([dataset_x.iloc[:, 0:EMBEDDING_LENGTH], chemical_dataset.iloc[:, 0:3]], ignore_index=True, axis=1)
big_chemical_dataset.head()
print(big_chemical_dataset.shape)

jump = 3

for i in range(1, 43):
    big_chemical_dataset = pd.concat([big_chemical_dataset, dataset_x.iloc[:, EMBEDDING_LENGTH * i: EMBEDDING_LENGTH * (i + 1)],
                                      chemical_dataset.iloc[:, jump:jump + 3]], ignore_index=True, axis=1)
    jump += 3

(770, 1027)


In [ ]:
big_chemical_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,44151,44152,44153,44154,44155,44156,44157,44158,44159,44160
0,-0.016333,0.030015,-0.111715,-0.139209,0.128241,0.073813,-0.042372,-0.076964,0.007215,0.098894,...,-0.106296,0.003818,0.004630,0.036363,-0.059417,-0.096753,0.115043,0.0,0.0,0.0
1,-0.088772,-0.064287,-0.058297,-0.213957,-0.001278,-0.030998,-0.202203,-0.236997,0.045164,0.157874,...,-0.126160,0.211296,-0.055563,-0.029729,0.175891,-0.076758,-0.008087,0.0,0.0,0.0
2,-0.010698,0.063155,-0.126877,-0.122769,0.094866,0.038334,-0.028891,-0.076412,-0.015086,0.100730,...,-0.119689,0.026145,0.016671,0.052288,-0.043501,-0.126030,0.078901,0.0,0.0,0.0
3,-0.010015,-0.167163,-0.048913,-0.241675,-0.153408,-0.030678,-0.204013,-0.248120,-0.003481,0.181270,...,-0.127472,0.176770,-0.020787,-0.171453,0.205280,-0.127785,-0.053986,0.0,0.0,0.0
4,0.004464,0.025306,-0.134299,-0.116793,0.108124,0.084962,-0.029363,-0.064316,-0.002252,0.084748,...,-0.104669,0.003502,0.012163,0.035501,-0.046423,-0.095274,0.108861,0.0,0.0,0.0


In [ ]:
43*771

33153

In [ ]:
# BERT + 2D-CNN Model (Average of Kast 4 Layers) + Chemical Properties + Hyperparameter Tuning

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(big_chemical_dataset, y_values, test_size = 0.10, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

def build_model_ch(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(filters=hp.Int("conv_1_filter", min_value=10, max_value=32, step=4),
            kernel_size=hp.Choice("conv_1_kernel", values=[3, 7]), activation = "relu", input_shape=(EMBEDDING_LENGTH + 3, 43, 1)),

        keras.layers.Conv2D(filters=hp.Int("conv_2_filter", min_value=32, max_value=64, step=4),
            kernel_size=hp.Choice("conv_2_kernel", values=[3, 6]), activation = "relu"),

        keras.layers.Flatten(),
        keras.layers.Dense(units=hp.Int("dense_1_units", min_value=32, max_value=80, step=4), activation = "relu"),
        keras.layers.Dense(2, activation = "softmax")
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 5e-5, 1e-5])),
                  loss = "categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.summary()
    return model

tuner_ch = RandomSearch(build_model_ch, objective = "val_accuracy", max_trials = 5, directory = "output", project_name = project_name)

# BERT_CH_HUMAN1_BEST_PARAMETERS -> Set1    1e-2, 1e-3, 1e-4, 5e-5, 1e-5
# BERT_CH_HUMAN1_BEST_PARAMETERS_SET2 -> Set2
# BERT_CH_HUMAN1_BEST_PARAMETERS_SET3 -> Set3

tuner_ch.search(np.asarray(x_train).reshape(len(np.asarray(x_train)),EMBEDDING_LENGTH + 3,43,1), utils.to_categorical(y_train,2),
              validation_data = (np.asarray(x_val.values.reshape(len(np.asarray(x_val)),EMBEDDING_LENGTH + 3,43,1)), utils.to_categorical(y_val,2)),
              epochs = 60)

model_2D_ht_ch = tuner_ch.get_best_models(num_models=1)[0]

In [ ]:
from sklearn.metrics import confusion_matrix

history4 = model_2D_ht_ch.fit(np.asarray(x_train).reshape(len(x_train), EMBEDDING_LENGTH + 3, 43, 1), utils.to_categorical(y_train, 2),
                          validation_data=(np.asarray(x_val.values.reshape(len(x_val), EMBEDDING_LENGTH + 3, 43, 1)), utils.to_categorical(y_val, 2)),
                          epochs=10, batch_size=20, verbose=1)

plt.plot(history4.history["accuracy"])
plt.plot(history4.history["val_accuracy"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.title("2D CNN Model Accuracy After Ch. Approach", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

plt.plot(history4.history["loss"])
plt.plot(history4.history["val_loss"])
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Loss", fontsize=10)
plt.title("2D CNN Model Loss After Ch. Approach", fontsize=15)
plt.legend(["Train", "Test"], loc="upper left", prop={"size": 10})
plt.show()

print(model_2D_ht_ch.summary())

cnn_predictions4 = model_2D_ht_ch.predict(np.asarray(x_test).reshape(len(np.asarray(x_test)), EMBEDDING_LENGTH + 3, 43, 1))
bert2_probs_ch = cnn_predictions4[:, 1]
cnn_predictions4 = np.argmax(cnn_predictions4, axis=1)

print("Model accuracy score: " + str(accuracy_score(y_test, cnn_predictions4)))
print("f1 score: ", metrics.f1_score(y_test, cnn_predictions4))
print("Precision:", metrics.precision_score(y_test, cnn_predictions4))
print("Recall:", metrics.recall_score(y_test, cnn_predictions4))

conf_matrix_ch = confusion_matrix(y_test, cnn_predictions4)
plt.figure(figsize = (8, 6))
sns.heatmap(conf_matrix_ch, annot=True, fmt="d", cbar=False)
plt.title("BERT + 2D CNN + H. Tuning Confusion Matrix", fontsize = 20)
plt.show()

print(cnn_predictions4)

fpr_keras_bert_cnn2_hyp_ch, tpr_keras_bert_cnn2_hyp_ch, thresholds_keras_ch = roc_curve(y_test, bert2_probs_ch)
auc_keras_bert_cnn2_hyp_ch = auc(fpr_keras_bert_cnn2_hyp_ch, tpr_keras_bert_cnn2_hyp_ch)
print("ROC AUC Score:", auc_keras_bert_cnn2_hyp_ch)

precision_bert2_ch, recall_bert2_ch, _ = precision_recall_curve(y_test, bert2_probs_ch)
auc_precision_recall_bert2_ch = auc(recall_bert2_ch, precision_bert2_ch)
print("Precision-Recall AUC Score:", auc_precision_recall_bert2_ch)

draw_roc_pr_curve(fpr_keras_bert_cnn2_hyp_ch, tpr_keras_bert_cnn2_hyp_ch, auc_keras_bert_cnn2_hyp_ch,
                  recall_bert2_ch, precision_bert2_ch, auc_precision_recall_bert2_ch,
                  model_name = MODEL_NAME + "+ 2D-CNN + Ch. Prop.")

## Comparison of the Results:

In [ ]:
# ROC AUC

plt.plot([0,1],[0,1], 'k--')

plt.plot(fpr_keras_dt, tpr_keras_dt, label="Decision Tree: {:.3f}".format(auc_keras_dt))
plt.plot(fpr_keras_rf, tpr_keras_rf, label="Random Forest: {:.3f}".format(auc_keras_rf))
plt.plot(fpr_keras_xgb, tpr_keras_xgb, label="XGBoost: {:.3f}".format(auc_keras_xgb))

plt.plot(fpr_keras_bert_rf, tpr_keras_bert_rf, label="BERT + Random Forest: {:.3f}".format(auc_keras_bert_rf))
plt.plot(fpr_keras_bert_xgb, tpr_keras_bert_xgb, label="BERT + XGBoost: {:.3f}".format(auc_keras_bert_xgb))
plt.plot(fpr_keras_bert_cnn1_hyp, tpr_keras_bert_cnn1_hyp, label="BERT + 1D-CNN: {:.3f}".format(auc_keras_bert_cnn1_hyp))
plt.plot(fpr_keras_bert_cnn2_hyp, tpr_keras_bert_cnn2_hyp, label="BERT2OMe: {:.3f}".format(auc_keras_bert_cnn2_hyp))
plt.plot(fpr_keras_bert_cnn2_hyp_ch, tpr_keras_bert_cnn2_hyp_ch, label="BERT2OMe with Chemical Properties: {:.3f}".format(auc_keras_bert_cnn2_hyp_ch))

plt.legend(loc=4, prop={'size': 8})
plt.xlabel("False Positive Rate", fontsize = 20)
plt.ylabel("True Positive Rate", fontsize = 20)
plt.title("ROC Curve", fontsize = 20)
plt.show()

print()

# PR AUC

plt.clf()
plt.plot([0,1],[0,1], 'k--')

plt.plot(recall_dt, precision_dt, label="Decision Tree: {:.3f}".format(auc_precision_recall_dt))
plt.plot(recall_rf, precision_rf, label="Random Forest: {:.3f}".format(auc_precision_recall_rf))
plt.plot(recall_xgb, precision_xgb, label="XGBoost: {:.3f}".format(auc_precision_recall_xgb))

plt.plot(recall_bert2_rf, precision_bert2_rf, label="BERT + Random Forest: {:.3f}".format(auc_precision_recall_bert2_rf))
plt.plot(recall_bert2_xgb, precision_bert2_xgb, label="BERT + XGBoost: {:.3f}".format(auc_precision_recall_bert2_xgb))
plt.plot(recall_bert2_cnn1, precision_bert2_cnn1, label="BERT + 1D-CNN: {:.3f}".format(auc_precision_recall_bert2_cnn1))
plt.plot(recall_bert2, precision_bert2, label="BERT2OMe: {:.3f}".format(auc_precision_recall_bert2))
plt.plot(recall_bert2_ch, precision_bert2_ch, label="BERT2OMe with Chemical Properties: {:.3f}".format(auc_precision_recall_bert2_ch))

plt.legend(loc=4, prop={'size': 8})
plt.xlabel("Recall", fontsize = 20)
plt.ylabel("Precision", fontsize = 20)
plt.title("PR Curve", fontsize = 20)
plt.show()